In [1]:
import selenium.webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
import json
import csv
import requests

C:\Users\janva\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")
driver.get("https://www.gall.nl/grolsch-lentebok-30cl-992739.html")

<ipython-input-3-2fa5cad6ae61>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")


In [4]:
res = driver.page_source.encode('utf-8')
soup = BeautifulSoup(res, "html.parser")

In [14]:
def get_soup(url):
    #driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")
    #driver.get(url)
    #res = driver.page_source.encode('utf-8')
    #soup = BeautifulSoup(res, "html.parser")
    r = requests.get(url, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(r.text, "html.parser")
    return soup

In [20]:
soup = get_soup("https://www.gall.nl/grolsch-lentebok-30cl-992739.html")

In [4]:
r = requests.get("https://www.gall.nl/grolsch-lentebok-30cl-992739.html", headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
soup = BeautifulSoup(r.text, "html.parser")
soup


<!DOCTYPE html>

<html class="no-js is--v2" data-edit-mode="false" dir="ltr" lang="nl">
<head>
<meta charset="utf-8"/>
<script src="https://cdn.optimizely.com/js/19212021254.js"></script>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="/on/demandware.static/Sites-gall-nl-Site/-/nl_NL/v1679387338702/js-external/cookie-consent.min.js" name="cookie-script"/>
<meta content="" name="client-log-level"/>
<script data-mopinion-deployment-id="2o8m5cnx2oxb82ji8ign87czqii5jwgsrwv" defer="" id="mopinion-script" src="//deploy.mopinion.com/js/pastease.js"></script>
<meta content="Grolsch Lentebok - 30CL kopen? | Gall &amp; Gall" property="og:title"/>
<meta content="product" property="og:type"/>
<meta content="https://static.gall.nl/images/IMG_684531_500.png?rev=0.2" property="og:image"/>
<meta content="https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Show?pid=992739" property="og:url"

In [15]:
soup = get_soup("https://www.gall.nl/lawsons-dry-hills-riesling-wit-75cl-121304.html")

In [21]:
if "url" in json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))[0]:
    print("yes")
else:
    print('no')

yes


In [22]:
data = []
reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))[0]
if "review" in reviews:
    for review in reviews['review']:
        info = {'link' : "asdf",
                "rating" : review['reviewRating']['ratingValue'],
                "date" : review['datePublished'],
                "review" : review['description']}
        try:
            info['type'] = soup.find('span', class_="pdp-info_profile").text.strip()
        except:
            info['type'] = None
        try:
            info['rating_average'] = float(soup.find('span', class_ ="rating_label").text)
        except:
            info['rating_average'] = None
        try:
            info['count'] = int(soup.find('span', class_ ="rating_label")['data-count'])
        except:
            info['count'] = None
        try:
            info['title'] = soup.find('h1', class_ ="pdp-info_name").text
        except:
            info['title'] = None
        try:
            info['country'] = soup.find('span', class_ = re.compile("flag-round u-flag-large u-hidden"))['title']
        except:
            info['country'] = None
        try:
            info['size'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[0].text
        except:
            info['size'] = None
        try:
            info['percentage'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[1].text
        except:
            info['percentage'] = None
        try:
            info['year'] = soup.find('p', class_ ="pdp-info_desc").find_all('span')[2].text
        except:
            info['year'] = None
        try:
            info['badge'] = soup.find('p', class_ ="pdp-info_badges").text.strip()
        except:
            info['badge'] = None
        try:
            info['price'] = soup.find('strong', class_ = "price-v2 pdp-info_price base-price")['aria-label'].replace('€ ', '')
        except:
            info['price'] = None
        try:
            info['main_description'] = soup.find('div', class_ ="pdp-info_short s-rich-text").text.strip()
        except:
            info['main_description'] = None
        try:
            usps = []
            for item in soup.find('ul', class_ = "m-usp-bar").find_all('div', class_="usp__text"):
                usps.append(item.text.strip())
            info['usps'] = usps
        except:
            info['usps'] = None

        try:
            info['secondary_description'] = soup.find('div', class_ = "pdp-details-description s-rich-text").text.strip()
        except:
            info['secondary_description'] = None
        details = [e.text for e in soup.find_all('span', class_ ="pdp-details__item-label")]
        try:
            pos = details.index("Smaak")
            info['taste'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['taste'] = None
        try:
            pos = details.index("Lekker bij")
            info['complementaries'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['complementaries'] = None
        try:
            pos = details.index("Druivensoort")
            info['grape'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['grape'] = None
        try:
            pos = details.index("Etiketinformatie")
            info['label'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['label'] = None
        try:
            pos = details.index("Prijswinnend")
            info['certificate'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
        except:
            info['certificate'] = None

        try:
            info['producer_description'] = soup.find('div', class_="background-story__description s-rich-text read-more__text").text.strip()
        except:
            info['producer_description'] = None
        try:
            info['producer'] = soup.find('span', class_="background-story__name").text
        except:
            info['producer'] = None
        data.append(info)
else:
    pass
data

[]

In [7]:
info.keys()

dict_keys(['link', 'rating', 'date', 'review', 'type', 'rating_average', 'count', 'title', 'country', 'size', 'percentage', 'year', 'badge', 'price', 'main_description', 'usps', 'secondary_description', 'taste', 'complementaries', 'grape', 'label', 'certificate', 'producer_description', 'producer'])

In [35]:
soup.find('p', class_ ="pdp-info_desc").find_all('span')

[<span class="pdp-info_country">
 <span class="fi fi-cl u-flag-round u-flag-large u-hidden" title="Chili"></span>
 Chili
 - Cabernet sauvignon
 </span>,
 <span class="fi fi-cl u-flag-round u-flag-large u-hidden" title="Chili"></span>,
 <span>2020</span>]

In [30]:
with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/no_reviews.csv", "w", encoding = 'utf-8-sig', newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(['link'])

In [28]:
def get_product_info(product_url, data):
    soup = get_soup(product_url)
    reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))[0]
    #Check if there are reviews
    if "review" in reviews:
        for review in reviews['review']:
            info = {'link' : product_url,
                    "rating" : review['reviewRating']['ratingValue'],
                    "date" : review['datePublished'],
                    "review" : review['description']}
            try:
                info['type'] = soup.find('span', class_="pdp-info_profile").text.strip()
            except:
                info['type'] = None
            try:
                info['rating_average'] = float(soup.find('span', class_ ="rating_label").text)
            except:
                info['rating_average'] = None
            try:
                info['count'] = int(soup.find('span', class_ ="rating_label")['data-count'])
            except:
                info['count'] = None
            try:
                info['title'] = soup.find('h1', class_ ="pdp-info_name").text
            except:
                info['title'] = None
            try:
                info['country'] = soup.find('span', class_ = re.compile("flag-round u-flag-large u-hidden"))['title']
            except:
                info['country'] = None
            try:
                info['size'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[0].text
            except:
                info['size'] = None
            try:
                info['percentage'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[1].text
            except:
                info['percentage'] = None
            try:
                info['year'] = soup.find('p', class_ ="pdp-info_desc").find_all('span')[2].text
            except:
                info['year'] = None
            try:
                info['badge'] = soup.find('p', class_ ="pdp-info_badges").text.strip()
            except:
                info['badge'] = None
            try:
                info['price'] = soup.find('strong', class_ = "price-v2 pdp-info_price base-price")['aria-label'].replace('€ ', '')
            except:
                info['price'] = None
            try:
                info['main_description'] = soup.find('div', class_ ="pdp-info_short s-rich-text").text.strip()
            except:
                info['main_description'] = None
            try:
                usps = []
                for item in soup.find('ul', class_ = "m-usp-bar").find_all('div', class_="usp__text"):
                    usps.append(item.text.strip())
                info['usps'] = usps
            except:
                info['usps'] = None

            try:
                info['secondary_description'] = soup.find('div', class_ = "pdp-details-description s-rich-text").text.strip()
            except:
                info['secondary_description'] = None
            details = [e.text for e in soup.find_all('span', class_ ="pdp-details__item-label")]
            try:
                pos = details.index("Smaak")
                info['taste'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['taste'] = None
            try:
                pos = details.index("Lekker bij")
                info['complementaries'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['complementaries'] = None
            try:
                pos = details.index("Druivensoort")
                info['grape'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['grape'] = None
            try:
                pos = details.index("Etiketinformatie")
                info['label'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['label'] = None
            try:
                pos = details.index("Prijswinnend")
                info['certificate'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
            except:
                info['certificate'] = None

            try:
                info['producer_description'] = soup.find('div', class_="background-story__description s-rich-text read-more__text").text.strip()
            except:
                info['producer_description'] = None
            try:
                info['producer'] = soup.find('span', class_="background-story__name").text
            except:
                info['producer'] = None
            try:
                data.append(info)
                if len(data) % 100 == 0:
                    print(len(data))
            except:
                print('could not append')

            with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews_corrected.csv", "a", encoding = 'utf-8-sig', newline = '') as csv_file:
                writer = csv.writer(csv_file, delimiter = ";")
                writer.writerow(list(info.values()))
    #Else there is no written review
    else:
        print(f"There are no reviews for {product_url}")
        with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/no_reviews.csv", "a", encoding = 'utf-8-sig', newline = '') as csv_file:
            writer = csv.writer(csv_file, delimiter = ";")
            writer.writerow([product_url])
        sleep(2)


In [17]:
data

In [9]:
reviews = json.loads(str(soup.find('script', {"type":"application/ld+json"}).contents[0].replace('\n', '')))

In [10]:
for review in reviews[0]['review']:
    info = {
    "rating" : review['reviewRating']['ratingValue'],
    "date" : review['datePublished'],
    "description" : review['description']}
    print(info)

{'rating': 5, 'date': '2023-02-10', 'description': 'Mooie wijn voor goede prijs. Licht.'}
{'rating': 4, 'date': '2023-02-08', 'description': 'Goede prijs, kwaliteit verhouding.'}
{'rating': 4, 'date': '2022-11-30', 'description': 'Is mijn huiswijn: met kaas e.d. lekkere wijn.'}
{'rating': 4, 'date': '2022-10-08', 'description': 'Prima, zoals vanouds zeer vertrouwd'}
{'rating': 4, 'date': '2022-10-05', 'description': 'Heerlijke Chileense wijn voor een zeer schappelijke prijs met een goede  smaak. Lekker bij kaas,tapas,pasta gerechten,of gewoon als borrelwijn'}
{'rating': 4, 'date': '2022-10-01', 'description': 'Mooie wijn.'}
{'rating': 4, 'date': '2022-09-24', 'description': 'Lekkere drinkbare wijn'}
{'rating': 5, 'date': '2022-05-04', 'description': 'Fijne smaak'}
{'rating': 4, 'date': '2022-04-23', 'description': 'Hele lekker wijn, voor bij rood vlees, barbecue en ook om zo te drinken.'}
{'rating': 4, 'date': '2022-04-19', 'description': 'Meerdere keren gekocht en nooit teleurgesteld.

In [12]:
data = []
get_product_info("https://www.gall.nl/valdivieso-winemakers-reserva-cabernet-sauvignon-rood-75cl-387568.html", data)
data

[{'link': 'https://www.gall.nl/valdivieso-winemakers-reserva-cabernet-sauvignon-rood-75cl-387568.html',
  'rating': 4.3,
  'date': '2023-02-10',
  'review': 'Mooie wijn voor goede prijs. Licht.',
  'type': 'Easy Rider',
  'count': 178,
  'title': "Valdivieso Winemaker's Reserva Cabernet Sauvignon",
  'country': 'Chili',
  'size': '75CL',
  'percentage': '13,5%',
  'year': '2020',
  'badge': None,
  'price': 7.98,
  'main_description': 'De Nieuw-Zeelandse wijnmaker Brett David Jackson zorgt jaar in jaar uit voor een soepele wijn met stevige tannines. Je proeft cassis, zwarte bessen en frambozen. Rond en zacht met specerijen.',
  'usps': ['Bestel voor 22:00 vandaag, morgen in huis',
   'Gratis bezorging vanaf € 50,-',
   'Gratis ophalen in een Gall & Gall winkel'],
  'secondary_description': None,
  'taste': 'rijp,cassisen houtrijping',
  'complementaries': 'rundvlees ,lamsvleesen kaas',
  'grape': 'Cabernet sauvignon',
  'label': 'Bevat sulfur dioxide and sulfits',
  'certificate': '2 k

In [24]:
product_pages = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/gall_product_links.csv", sep = ";", index_col = 0)
product_pages

,product,page_url,name,product_link,type,country,badge,price,size,rating,count,year
0,1,https://www.gall.nl/wijn/?start=0,Mucho Más Tinto,https://www.gall.nl/mucho-mas-tinto-rood-75cl-...,Easy Rider,Spanje,2e halve prijs,6.99,75CL,4.6,200.0,NaN
1,2,https://www.gall.nl/wijn/?start=0,Mucho Más Blanco,https://www.gall.nl/mucho-mas-blanco-wit-75cl-...,Fruit Lover,Spanje,2e halve prijs,6.99,75CL,4.4,199.0,NaN
2,3,https://www.gall.nl/wijn/?start=0,La Palma Chardonnay,https://www.gall.nl/la-palma-chardonnay-wit-75...,Big White,Chili,2e halve prijs,5.99,75CL,4.4,200.0,2022.0
3,4,https://www.gall.nl/wijn/?start=0,Ogio Pinot Grigio,https://www.gall.nl/ogio-pinot-grigio-wit-75cl...,Fruit Lover,Italië,2e halve prijs,6.59,75CL,4.1,188.0,2022.0
4,5,https://www.gall.nl/wijn/?start=0,Valdivieso Merlot,https://www.gall.nl/valdivieso-merlot-rood-75c...,Easy Rider,Chili,2e halve prijs,6.59,75CL,4.3,199.0,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1794,1795,https://www.gall.nl/wijn/?start=1788,Connétable de Talbot,https://www.gall.nl/connetable-de-talbot-rood-...,Royal Red,Frankrijk,2e halve prijs,32.99,75CL,NaN,NaN,2018.0
1795,1796,https://www.gall.nl/wijn/?start=1788,Umani Ronchi Cúmaro Conero Riserva,https://www.gall.nl/umani-ronchi-c%C3%BAmaro-c...,Classic Red,Italië,2e halve prijs,30.59,75CL,NaN,NaN,2016.0
1796,1797,https://www.gall.nl/wijn/?start=1800,Penfolds RWT Barossa Valley Shiraz,https://www.gall.nl/penfolds-rwt-barossa-valle...,Big Red,Australië,NaN,163.19,75CL,NaN,NaN,2016.0
1797,1798,https://www.gall.nl/wijn/?start=1800,Château Barde-Haut,https://www.gall.nl/ch%C3%A2teau-barde-haut-ro...,Royal Red,Frankrijk,2e halve prijs,54.49,75CL,NaN,NaN,2016.0


In [55]:
no_reviews = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/no_reviews.csv", sep = ";")
no_reviews

,link
0,https://www.gall.nl/lawsons-dry-hills-riesling...
1,https://www.gall.nl/errazuriz-max-merlot-rood-...
2,https://www.gall.nl/dom%C3%A4ne-wachau-chardon...
3,https://www.gall.nl/warwick-wine-estate-first-...
4,https://www.gall.nl/eagles-nest-merlot-rood-75...
5,https://www.gall.nl/boschkloof-sauvignon-blanc...
6,https://www.gall.nl/villa-locatelli-merlot-roo...


In [57]:
data_so_far = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews_corrected.csv", sep = ";")
data_so_far
links_so_far = data_so_far['link'].unique()
links_to_do = [link for link in product_pages['product_link'] if link not in links_so_far]
print(len(product_pages))
print(len(links_so_far))
print(len(links_to_do))
print(len(no_reviews))

1799
143
1656
7


In [59]:
data = []
i = 0
#driver = selenium.webdriver.Chrome(executable_path="C:/Users/janva/Downloads/chromedriver.exe")
for product_link in links_to_do:
    if product_link not in list(no_reviews['link']):
        print(product_link)
        if i % 100 == 0:
            print(round(i/len(links_to_do),4)*100)
        i += 1
        try:
            get_product_info(product_link, data)
        except:
            print(f"unknown error for {product_link}")
            with open("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/no_reviews.csv", "a", encoding = 'utf-8-sig', newline = '') as csv_file:
                writer = csv.writer(csv_file, delimiter = ";")
                writer.writerow([product_link])
    else:
        print(f"already looked at {product_link}, there are no reviews")

already looked at https://www.gall.nl/lawsons-dry-hills-riesling-wit-75cl-121304.html, there are no reviews
already looked at https://www.gall.nl/errazuriz-max-merlot-rood-75cl-414344.html, there are no reviews
already looked at https://www.gall.nl/dom%C3%A4ne-wachau-chardonnay-federspiel-terrassen-wit-75cl-107611.html, there are no reviews
already looked at https://www.gall.nl/warwick-wine-estate-first-lady-red-rood-75cl-110884.html, there are no reviews
already looked at https://www.gall.nl/eagles-nest-merlot-rood-75cl-381497.html, there are no reviews
already looked at https://www.gall.nl/boschkloof-sauvignon-blanc-wit-75cl-391476.html, there are no reviews
already looked at https://www.gall.nl/villa-locatelli-merlot-rood-75cl-258881.html, there are no reviews
https://www.gall.nl/schott-zwiesel-rode-wijnglazen-%28set-van-2%29-2st-908630.html
0.0
unknown error for https://www.gall.nl/schott-zwiesel-rode-wijnglazen-%28set-van-2%29-2st-908630.html
https://www.gall.nl/torres-altos-iberi

There are no reviews for https://www.gall.nl/bodega-garzon-reserva-cabernet-franc-rood-75cl-106623.html
https://www.gall.nl/bodega-garzon-estate-pinot-grigio-wit-75cl-106534.html
https://www.gall.nl/errazuriz-aconcagua-costa-chardonnay-wit-75cl-414328.html
There are no reviews for https://www.gall.nl/errazuriz-aconcagua-costa-chardonnay-wit-75cl-414328.html
https://www.gall.nl/lawsons-dry-hills-pinot-noir-rood-75cl-121002.html
https://www.gall.nl/arrogant-frog-rose-bag-in-box-rose-300cl-138711.html
1900
https://www.gall.nl/errazuriz-aconcagua-alto-carmenere-rood-75cl-409588.html
There are no reviews for https://www.gall.nl/errazuriz-aconcagua-alto-carmenere-rood-75cl-409588.html
https://www.gall.nl/pierre-de-prunet-rose-rose-75cl-129429.html
2000
https://www.gall.nl/raymond-r-collection-cabernet-sauvignon-rood-75cl-394149.html
6.04
https://www.gall.nl/undurraga-t.h.-pinot-noir-leyda-rood-75cl-119318.html
https://www.gall.nl/angoris-sauvignon-blanc-wit-75cl-258377.html
There are no revi

https://www.gall.nl/casal-mendes-vinho-verde-wit-75cl-353027.html
https://www.gall.nl/valdivieso-single-vineyard-syrah-rood-75cl-110191.html
https://www.gall.nl/karl-kaspar-auslese-wit-75cl-219991.html
https://www.gall.nl/maison-ventenac-reserve-de-jeanne-rood-75cl-143022.html
There are no reviews for https://www.gall.nl/maison-ventenac-reserve-de-jeanne-rood-75cl-143022.html
https://www.gall.nl/nice-rose-rose-75cl-409340.html
3100
https://www.gall.nl/lautus-savvy-red-rood-75cl-404195.html
https://www.gall.nl/ch%C3%A2teau-de-berne-romance-rose-75cl-163759.html
https://www.gall.nl/cune-seleccion-de-fincas-crianza-rood-75cl-402133.html
https://www.gall.nl/bodega-garzon-reserva-marselan-rood-75cl-106631.html
There are no reviews for https://www.gall.nl/bodega-garzon-reserva-marselan-rood-75cl-106631.html
https://www.gall.nl/lawsons-dry-hills-reserve-pinot-noir-rood-75cl-124133.html
There are no reviews for https://www.gall.nl/lawsons-dry-hills-reserve-pinot-noir-rood-75cl-124133.html
http

https://www.gall.nl/deloach-california-pinot-noir-rood-75cl-112194.html
There are no reviews for https://www.gall.nl/deloach-california-pinot-noir-rood-75cl-112194.html
https://www.gall.nl/mirabeau-classic-rose-geschenkverpakking-rose-75cl-117781.html
https://www.gall.nl/freetime-merlot-rood-75cl-117684.html
https://www.gall.nl/andrian-chardonnay-somereto-wit-75cl-110574.html
There are no reviews for https://www.gall.nl/andrian-chardonnay-somereto-wit-75cl-110574.html
https://www.gall.nl/araia-soave-classico-wit-75cl-259721.html
https://www.gall.nl/claus-de-peyron-vacqueyras-rood-75cl-189650.html
https://www.gall.nl/diego-de-almagro-valdepenas-rood-75cl-120022.html
https://www.gall.nl/love-by-leoube-rose-rose-75cl-163295.html
https://www.gall.nl/torres-coronas-rood-75cl-313068.html
https://www.gall.nl/errazuriz-max-cabernet-sauvignon-rood-75cl-409359.html
https://www.gall.nl/vi%C3%B1a-real-gran-reserva-rood-75cl-411116.html
https://www.gall.nl/gruber-roschitz-gr%C3%BCner-veltliner-terr

https://www.gall.nl/gran-sasso-montepulciano-dabruzzo-rood-75cl-412600.html
https://www.gall.nl/jc-boisset-m%C3%A2con-ige-wit-75cl-169641.html
https://www.gall.nl/vidal-fleury-c%C3%B4tes-du-rh%C3%B4ne-magnum-rood-150cl-110183.html
https://www.gall.nl/kuentz-bas-trois-chateaux-auxerrois-wit-75cl-144134.html
There are no reviews for https://www.gall.nl/kuentz-bas-trois-chateaux-auxerrois-wit-75cl-144134.html
https://www.gall.nl/contino-gran-reserva-rood-75cl-123579.html
There are no reviews for https://www.gall.nl/contino-gran-reserva-rood-75cl-123579.html
https://www.gall.nl/raymond-reserve-selection-cabernet-sauvignon-rood-75cl-394130.html
There are no reviews for https://www.gall.nl/raymond-reserve-selection-cabernet-sauvignon-rood-75cl-394130.html
https://www.gall.nl/maritima-rose-75cl-412570.html
https://www.gall.nl/crasto-vinho-branco-wit-75cl-359971.html
There are no reviews for https://www.gall.nl/crasto-vinho-branco-wit-75cl-359971.html
https://www.gall.nl/kurf%C3%BCrst-karl-kas

...

https://www.gall.nl/argiolas-serra-lori-rosato-rose-75cl-257079.html
84.54
There are no reviews for https://www.gall.nl/argiolas-serra-lori-rosato-rose-75cl-257079.html
https://www.gall.nl/kanonkop-pinotage-magnum-rood-150cl-141070.html
There are no reviews for https://www.gall.nl/kanonkop-pinotage-magnum-rood-150cl-141070.html
https://www.gall.nl/ch%C3%A2teau-german-marbuzet-rood-75cl-415316.html
There are no reviews for https://www.gall.nl/ch%C3%A2teau-german-marbuzet-rood-75cl-415316.html
https://www.gall.nl/tasca-dalmerita-lam%C3%B9ri-nero-davola-rood-75cl-256943.html
There are no reviews for https://www.gall.nl/tasca-dalmerita-lam%C3%B9ri-nero-davola-rood-75cl-256943.html
https://www.gall.nl/laurus-aoc-hermitage-blanc-wit-75cl-131148.html
There are no reviews for https://www.gall.nl/laurus-aoc-hermitage-blanc-wit-75cl-131148.html
https://www.gall.nl/ch%C3%A2teau-gloria-rood-75cl-144460.html
There are no reviews for https://www.gall.nl/ch%C3%A2teau-gloria-rood-75cl-144460.html
http

https://www.gall.nl/wijnbox-barolo-rood-3x75cl-171514.html
There are no reviews for https://www.gall.nl/wijnbox-barolo-rood-3x75cl-171514.html
https://www.gall.nl/brocard-chablis-fourchaume-wit-75cl-150797.html
There are no reviews for https://www.gall.nl/brocard-chablis-fourchaume-wit-75cl-150797.html
https://www.gall.nl/wijnbox-wereldse-wijnen-gemixt-gemengd-6x75cl-171247.html
https://www.gall.nl/ch%C3%A2teau-la-tour-carnet-rood-75cl-143405.html
https://www.gall.nl/cloudy-bay-sauvignon-blanc-wit-75cl-404586.html
https://www.gall.nl/argiolas-costamolino-vermentino-di-sardegna-wit-75cl-258237.html
https://www.gall.nl/chablisienne-chablis-fourchaume-wit-75cl-405078.html
There are no reviews for https://www.gall.nl/chablisienne-chablis-fourchaume-wit-75cl-405078.html
https://www.gall.nl/trimbach-riesling-cuvee-frederic-emile-wit-75cl-123323.html
https://www.gall.nl/groot-constantia-grand-constance-wit-37.5cl-469963.html
https://www.gall.nl/louis-bernard-ch%C3%A2teauneuf-du-pape-rood-75cl

There are no reviews for https://www.gall.nl/mathilde-chapoutier-rueda-wit-75cl-111651.html
https://www.gall.nl/le-cloitre-de-prieure-lichine-rood-75cl-114693.html
There are no reviews for https://www.gall.nl/le-cloitre-de-prieure-lichine-rood-75cl-114693.html
https://www.gall.nl/tenimenti-dalessandro-bosco-syrah-rood-75cl-100447.html
There are no reviews for https://www.gall.nl/tenimenti-dalessandro-bosco-syrah-rood-75cl-100447.html
https://www.gall.nl/le-vigne-di-zam%C3%B2-refosco-rood-75cl-258431.html
There are no reviews for https://www.gall.nl/le-vigne-di-zam%C3%B2-refosco-rood-75cl-258431.html
https://www.gall.nl/joseph-drouhin-chablis-premier-cru-secher-wit-75cl-169919.html
There are no reviews for https://www.gall.nl/joseph-drouhin-chablis-premier-cru-secher-wit-75cl-169919.html
https://www.gall.nl/ch%C3%A2teau-chasse-spleen-rood-75cl-134872.html
https://www.gall.nl/jc-boisset-pommard-les-arvelets-rood-75cl-122823.html
https://www.gall.nl/mazzei-castello-fonterutoli-rood-75cl-2

https://www.gall.nl/librandi-le-passule-passito-wit-50cl-111082.html
There are no reviews for https://www.gall.nl/librandi-le-passule-passito-wit-50cl-111082.html
https://www.gall.nl/ch%C3%A2teau-mouton-rothschild-rood-75cl-144487.html
There are no reviews for https://www.gall.nl/ch%C3%A2teau-mouton-rothschild-rood-75cl-144487.html
https://www.gall.nl/joseph-drouhin-nuits-saint-georges-vaucrains-rood-75cl-122580.html
There are no reviews for https://www.gall.nl/joseph-drouhin-nuits-saint-georges-vaucrains-rood-75cl-122580.html
https://www.gall.nl/catena-zapata-malbec-argentino-magnum-rood-150cl-125040.html
There are no reviews for https://www.gall.nl/catena-zapata-malbec-argentino-magnum-rood-150cl-125040.html
https://www.gall.nl/mazzei-mix-36-chianti-classico-rood-75cl-107018.html
There are no reviews for https://www.gall.nl/mazzei-mix-36-chianti-classico-rood-75cl-107018.html
https://www.gall.nl/antinori-tignanello-rood-75cl-256633.html
https://www.gall.nl/ch%C3%A2teau-la-fleur-de-ga

There are no reviews for https://www.gall.nl/umani-ronchi-jorio-montepulciano-rood-75cl-414662.html
https://www.gall.nl/umani-ronchi-san-lorenzo-rosso-rood-75cl-414654.html
There are no reviews for https://www.gall.nl/umani-ronchi-san-lorenzo-rosso-rood-75cl-414654.html
https://www.gall.nl/umani-ronchi-le-busche-chardonnay-wit-75cl-414603.html
There are no reviews for https://www.gall.nl/umani-ronchi-le-busche-chardonnay-wit-75cl-414603.html
https://www.gall.nl/zuccardi-los-oscuros-malbec-rood-75cl-414131.html
There are no reviews for https://www.gall.nl/zuccardi-los-oscuros-malbec-rood-75cl-414131.html
https://www.gall.nl/south-point-pinot-noir-rood-75cl-412376.html
There are no reviews for https://www.gall.nl/south-point-pinot-noir-rood-75cl-412376.html
https://www.gall.nl/alambrado-cabernet-franc-rood-75cl-413941.html
There are no reviews for https://www.gall.nl/alambrado-cabernet-franc-rood-75cl-413941.html
https://www.gall.nl/canallas-rose-rose-75cl-414972.html
There are no review

# Concationation

In [25]:
beer = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_beergall_with_reviews_corrected.csv", sep = ";")
wine = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/data_gall_with_reviews_corrected.csv", sep = ";")

In [28]:
beer['category'] = "beer"
wine['category'] = "wine"

In [30]:
print(len(wine))
print(len(beer))
concat_data = pd.concat([wine,beer])

825
311


In [31]:
concat_data.to_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/full_data_set.csv", sep = ";")